In [9]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import torch

from qecdec import RotatedSurfaceCode_Memory
from learned_decoders import *
from train_utils import *
from datetime import datetime
import pickle

d = 5
rounds = 5
p = 0.01
num_bp_iters = 5
learning_rate = 0.002
loss_beta = 1.0
train_offset = False
train_nf = True

In [2]:
expmt = RotatedSurfaceCode_Memory(
    d=d,
    rounds=rounds,
    basis='Z',
    data_qubit_error_rate=p,
    meas_error_rate=p,
)
print("Number of error mechanisms:", expmt.num_error_mechanisms)
print("Number of detectors:", expmt.num_detectors)
print("Number of observables:", expmt.num_observables)

Number of error mechanisms: 186
Number of detectors: 72
Number of observables: 1


In [ ]:
train_dataset, val_dataset = build_datasets(
    expmt,
    train_shots=10_000,
    val_shots=1_000,
    seed=42,
)
print("Size of train_dataset:", len(train_dataset))
print("Size of val_dataset:", len(val_dataset))

model = LearnedDMemOffNormBP(expmt.chkmat, expmt.prior, num_iters=num_bp_iters, train_offset=train_offset, train_nf=train_nf)
loss_fn = DecodingLoss(expmt.chkmat, expmt.obsmat, beta=loss_beta)
metric = DecodingMetric(expmt.chkmat, expmt.obsmat)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Size of train_dataset: 25556
Size of val_dataset: 815


In [4]:
train_gamma(
    model,
    train_dataset,
    val_dataset,
    loss_fn,
    metric,
    optimizer,
    num_epochs=20,
    batch_size=256,
    device="cpu",
    scheduler_kwargs=dict(factor=0.2, patience=3, threshold=1e-3, threshold_mode="abs"),
    early_stopper=EarlyStopper(patience=5, min_delta=1e-3),
)

Using cpu device


Epoch 1/20: 100%|██████████| 100/100 [07:02<00:00,  4.22s/it, avg_loss=0.706679, grad_norm=0.165694]


Epoch 1 Summary:
  Avg Train Loss: 0.706679
  Avg Val Loss: 0.736395
  wrong_syndrome_rate: 0.047853
  wrong_observable_rate: 0.006135
  failure_rate: 0.047853
  Learning Rate: 0.002000



Epoch 2/20: 100%|██████████| 100/100 [06:42<00:00,  4.02s/it, avg_loss=0.606926, grad_norm=0.134927]


Epoch 2 Summary:
  Avg Train Loss: 0.606926
  Avg Val Loss: 0.685253
  wrong_syndrome_rate: 0.041718
  wrong_observable_rate: 0.006135
  failure_rate: 0.041718
  Learning Rate: 0.002000



...

Epoch 18 Summary:
  Avg Train Loss: 0.538267
  Avg Val Loss: 0.638609
  wrong_syndrome_rate: 0.026994
  wrong_observable_rate: 0.008589
  failure_rate: 0.026994
  Learning Rate: 0.002000



Epoch 19/20: 100%|██████████| 100/100 [07:12<00:00,  4.33s/it, avg_loss=0.538015, grad_norm=0.130692]


Epoch 19 Summary:
  Avg Train Loss: 0.538015
  Avg Val Loss: 0.638925
  wrong_syndrome_rate: 0.025767
  wrong_observable_rate: 0.007362
  failure_rate: 0.025767
  Learning Rate: 0.002000



Epoch 20/20: 100%|██████████| 100/100 [07:11<00:00,  4.31s/it, avg_loss=0.537913, grad_norm=0.128910]


Epoch 20 Summary:
  Avg Train Loss: 0.537913
  Avg Val Loss: 0.639143
  wrong_syndrome_rate: 0.023313
  wrong_observable_rate: 0.007362
  failure_rate: 0.023313
  Learning Rate: 0.000400



In [10]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

gamma = model.gamma.detach().numpy().astype(np.float64)
np.save(f"learned_params/dmemoffnormbp_{now}_gamma.npy", gamma)

if train_offset:
    offset = [x.detach().tolist() for x in model.offset] # list[list[float]]
    with open(f"learned_params/dmemoffnormbp_{now}_offset.pkl", "wb") as f:
        pickle.dump(offset, f)

if train_nf:
    nf = [x.detach().tolist() for x in model.nf] # list[list[float]]
    with open(f"learned_params/dmemoffnormbp_{now}_nf.pkl", "wb") as f:
        pickle.dump(nf, f)
